In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
actions = np.array(['hello','i love you','my','Name','thanks','What','Your'])
DATA_PATH = os.path.join('MP_Data')
sequence_length = 30

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [4]:
label_map = {label:num for num, label in enumerate(actions)}

In [5]:
sequences, labels = [], []

for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [6]:
X = np.array(sequences)

In [7]:
np.array(sequences).shape

(210, 30, 1662)

In [8]:
y = to_categorical(labels).astype(int)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard

In [11]:
# log_dir = os.path.join('Logs')
# tb_callback = TensorBoard(log_dir=log_dir)
# Define log directory
log_dir = os.path.join('Logs')

# Define TensorBoard callback
tb_callback = TensorBoard(log_dir=log_dir)

In [12]:


# Define the CNN model
model = Sequential()

# Add Convolutional layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(30, 1662)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten layer to connect convolutional layers to dense layers
model.add(Flatten())

# Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output layer
model.add(Dense(actions.shape[0], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks (if needed)
tensorboard = TensorBoard(log_dir='logs')

# Train the model
# model.fit(X_train, y_train, epochs=..., batch_size=..., callbacks=[tensorboard])


c:\Users\prana\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [13]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [14]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - categorical_accuracy: 0.1293 - loss: 2.0017
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - categorical_accuracy: 0.3783 - loss: 1.7532
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - categorical_accuracy: 0.3014 - loss: 1.6004
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - categorical_accuracy: 0.3803 - loss: 1.5794
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - categorical_accuracy: 0.4041 - loss: 1.5203
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - categorical_accuracy: 0.4342 - loss: 1.3969
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - categorical_accuracy: 0.4342 - loss: 1.3229
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - categorical_accuracy: 0.4890 - loss: 1.2155
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - categorical_accuracy: 0.5445 - loss: 1.0782
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - categorical_accuracy: 0.5967 - loss: 1.0521
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0

In [15]:
model.summary

<bound method Model.summary of <Sequential name=sequential, built=True>>

In [16]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step


In [17]:
model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


array([[9.5619281e-09, 1.4248055e-03, 2.3064718e-05, 5.0330766e-09,
        1.4683954e-09, 1.3751831e-07, 9.9855191e-01],
       [1.5068679e-09, 4.3171351e-09, 7.7041209e-04, 9.9892080e-01,
        2.9708016e-09, 3.0878602e-04, 3.0280542e-10],
       [7.9360116e-01, 4.3279125e-13, 2.3801856e-07, 7.0692970e-08,
        2.0639734e-01, 1.4313304e-08, 1.2125148e-06],
       [1.4899301e-09, 3.5024485e-08, 9.7434968e-01, 4.5403947e-05,
        1.8829214e-04, 2.5394386e-02, 2.2148852e-05],
       [3.9307491e-10, 3.2368861e-11, 9.9971741e-01, 2.0228169e-06,
        2.5357780e-04, 2.6864691e-05, 9.8738482e-08],
       [9.1510559e-09, 7.0466216e-05, 3.0123259e-04, 1.0975054e-07,
        1.4200492e-08, 2.5766632e-05, 9.9960238e-01],
       [1.6166418e-07, 1.0192002e-08, 8.7871623e-01, 1.2128127e-01,
        9.7031165e-09, 2.2722129e-06, 1.7165271e-10],
       [1.9218533e-08, 1.7959042e-01, 7.9479258e-05, 1.2672444e-07,
        9.8116910e-09, 9.9002347e-05, 8.2023096e-01],
       [1.8892248e-08, 2

In [18]:
actions[np.argmax(res[3])]

'my'

In [19]:
np.sum(res[2])

1.0

In [20]:
actions[np.argmax(y_test[2])]

'hello'

In [21]:
actions[y_test[2]]

array(['i love you', 'hello', 'hello', 'hello', 'hello', 'hello', 'hello'],
      dtype='<U10')

In [22]:
actions[np.argmax(y_test[2])]

'hello'

In [23]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [24]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [25]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)
accuracy_score(ytrue, yhat)

0.9090909090909091

In [27]:
model.save('action.h5')

In [83]:
del model

NameError: name 'model' is not defined